### get_season_data

In [9]:
%run import_modules.ipynb
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = api_key
configuration.api_key_prefix['Authorization']='Bearer'
api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))
%run process_game.ipynb
%run extract_team_data.ipynb
%run fetch_data_with_retries.ipynb

In [8]:
# Function to get season data
def get_season_data(year, start_week, end_week):
    """
    Retrieves and processes data for a whole season.

    :param year: Year of the game data.
    :param start_week: The first week of the season.
    :param end_week: The last week of the season.
    :return: DataFrame containing extracted team statistics for the whole season.
    """
    all_data = []

    # Function to process data for a specific week
    def process_week(week):
        print(f"Processing week {week} of {year}")
        def api_call():
            return api_instance.get_team_game_stats(year=year, week=week)

        try:
            game_data = fetch_data_with_retries(api_call)
            week_data = extract_team_data(game_data, week)
            return week_data
        except Exception as e:
            print(f"An error occurred while processing week {week}: {e}")
            return pd.DataFrame()  # Return an empty DataFrame in case of failure
    
    # Use parallel processing to handle multiple weeks
    with ThreadPoolExecutor(max_workers=5) as executor:  # Adjust max_workers as needed
        futures = [executor.submit(process_week, week) for week in range(start_week, end_week + 1)]
        for future in as_completed(futures):
            try:
                week_data = future.result()
                if not week_data.empty:
                    all_data.append(week_data)
            except Exception as e:
                print(f"An error occurred: {e}")

    # Combine all data into a single DataFrame
    combined_df = pd.concat(all_data, ignore_index=True).fillna(0)  # Replace NaNs with 0 if needed
    return combined_df

NameError: name 'yr23' is not defined

In [7]:
teamStats20=get_season_data(year=2020,start_week=1,end_week=7)

Processing week 1 of 2020
Processing week 2 of 2020
Processing week 3 of 2020
Processing week 4 of 2020
Processing week 5 of 2020
Processing week 6 of 2020
Processing week 7 of 2020


In [6]:
teamStats20.head()

,game_id,team_id,team_name,home_away,points,week,tacklesForLoss,defensiveTDs,tackles,sacks,...,yardsPerPass,completionAttempts,netPassingYards,totalYards,fourthDownEff,thirdDownEff,firstDowns,puntReturnYards,puntReturnTDs,puntReturns
0,401239884,2617,Stephen F. Austin,away,14,1,5,0,38,1,...,6.3,14-21,133,230,1-1,3-8,14,0,0,0
1,401239884,2638,UTEP,home,24,1,5,0,25,1,...,7.6,17-28,212,364,1-2,8-13,19,0,0,0
2,401234576,2426,Navy,home,3,1,4,0,33,0,...,3.8,4-8,30,149,0-2,2-12,7,12,0,1
3,401234576,252,BYU,away,55,1,8,0,32,5,...,12.7,16-22,279,580,1-2,6-11,28,-2,0,1
4,401235700,349,Army,home,42,1,6,1,31,1,...,7.0,2-4,28,368,1-1,13-15,24,0,0,0


In [23]:
# con=sqlite3.connect('collegeFootball.db')
# teamStats20.to_sql(name='teamStats20',con=con)
# con.commit()
# con.close()
# con=sqlite3.connect('collegeFootball.db')
# p3=pd.read_sql("SELECT * FROM teamStats20",con=con)